Important click through this to clean the data and get the instruction data before running the other notebooks.


In [17]:
#Imports
import os
import shutil
import random
import json
import re
import ast
from PIL import Image

In [30]:
# Paths please change these to your own paths
feed = r"E:\uni\BA\data\input\feed"
feed_pictures = r"E:\uni\BA\data\input\feedPictures"
feed_latin = r"E:\uni\BA\data\input\feedLatin"
feed_text = r"E:\uni\BA\data\input\feedText"
output = r"E:\uni\BA\data\output"
path_box = r"E:\uni\BA\data\input\box"



Put fill up the prior defined folders with the apporopriate initial files:  
Add all the entities into the feed folder   
All the pictures into the feed_pictures folder    
All the files with the OCR result including the boxes in the box folder   
Make sure, that the output folder is empty.

In [19]:
def clean_price(price):
  # Remove any non-numeric characters except the decimal point
  cleaned_price = ''.join(char for char in price if char.isdigit() or char == '.')
    
  # Ensure it has two decimal places
  if '.' in cleaned_price:
    integer_part, decimal_part = cleaned_price.split('.')
    decimal_part = decimal_part[:2].ljust(2, '0')  # Ensure two decimal places
    corrected_price = f"{integer_part}.{decimal_part}"
  else:
    corrected_price = f"{cleaned_price}.00"  # No decimal point, add ".00"
    
  return corrected_price

In [20]:
#This function is used to clean the prices. Move the entities you want to have cleaned into your feed folder
def correct_price_format():

  folder_path = feed
  # Iterate over all files in the specified folder
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    with open(file_path, 'r') as file:
      data = json.load(file)
            
    # Check if the 'total' field is formatted correctly
    if 'total' in data:
      original_price = data['total']
      corrected_price = clean_price(original_price)

      if original_price != corrected_price:
        print(f"Correcting price in file {filename}: {original_price} -> {corrected_price}")
        data['total'] = corrected_price

                    # Write the corrected data back to the file
    with open(file_path, 'w') as file:
      json.dump(data, file, indent=4)
correct_price_format()

LATIN helper

In [21]:
def to_prompt(scan, img_size) -> str:
    # Convert ocr bboxes to latin boxes
    w, h = img_size
    texts = [x['text'] for x in scan]
    boxes = [x['bbox'] for x in scan]
      
    
    # Now continue with the latin prompting from https://github.dev/WenjinW/LATIN-Prompt
    line_boxes = []
    line_texts = []
    max_line_char_num = 0
    line_width = 0

    while len(boxes) > 0:
        line_box = [boxes.pop(0)]
        line_text = [texts.pop(0)]
        char_num = len(line_text[-1])
        line_union_box = line_box[-1]
        while len(boxes) > 0 and _is_same_line(line_box[-1], boxes[0]):
            line_box.append(boxes.pop(0))
            line_text.append(texts.pop(0))
            char_num += len(line_text[-1])
            line_union_box = _union_box(line_union_box, line_box[-1])
        line_boxes.append(line_box)
        line_texts.append(line_text)
        if char_num >= max_line_char_num:
            max_line_char_num = char_num
            line_width = line_union_box[2] - line_union_box[0]

    max_line_char_num = max(max_line_char_num, 1)
    char_width = line_width / max_line_char_num
    if char_width == 0:
        char_width = 1

    space_line_texts = []
    for i, line_box in enumerate(line_boxes):
        space_line_text = ""
        for j, box in enumerate(line_box):
            left_char_num = int(box[0] / char_width)
            space_line_text += " " * (left_char_num - len(space_line_text))
            space_line_text += line_texts[i][j]
        space_line_texts.append(space_line_text)

    return "\n".join(space_line_texts)


def _is_same_line(box1, box2):
    """
    Params:
        box1: [x1, y1, x2, y2]
        box2: [x1, y1, x2, y2]
    """
    box1_midy = (box1[1] + box1[3]) / 2
    box2_midy = (box2[1] + box2[3]) / 2

    if (
        box1_midy < box2[3]
        and box1_midy > box2[1]
        and box2_midy < box1[3]
        and box2_midy > box1[1]
    ):
        return True
    else:
        return False


def _union_box(box1, box2):
    """
    Params:
        box1: [x1, y1, x2, y2]
        box2: [x1, y1, x2, y2]
    """
    x1 = min(box1[0], box2[0])
    y1 = min(box1[1], box2[1])
    x2 = max(box1[2], box2[2])
    y2 = max(box1[3], box2[3])

    return [x1, y1, x2, y2]

In [22]:
def split_after_eighth_comma(line, filename):
    # Find the position of the eighth comma
    comma_positions = [pos for pos, char in enumerate(line) if char == ","]
    if len(line) == 0:
        return [-1,-1,-1,-1], ""
    if len(comma_positions) < 8:
        print(f"Error in file {filename}: The string does not contain eight commas.")
        print(line)
        raise ValueError("The string does not contain eight commas.")
    
    # Split the string at the position after the eighth comma
    split_position = comma_positions[7] + 1  # Add 1 to include the eighth comma
    
    # Extract the bbox and text parts
    bbox = line[:split_position - 1]  # Everything before the eighth comma
    text = line[split_position:]  # Everything after the eighth comma
    
    # Split the bbox into individual numbers
    bbox_numbers = bbox.split(",")
    
    # Select the first two and last two numbers
    selected_bbox = bbox_numbers[:2] + bbox_numbers[4:6]
    selected_bbox = [int(num) for num in selected_bbox]
    return selected_bbox, text

Make sure, that the output folder is empty, before running.   
While running, it is expected to fail, if a box file includes a not utf-8 encoded char. Then navigate to the box folder and search for the last printed name and remove the mentioned characters, before rerunning the function.

In [25]:
def latin_runner():
    
    image_files = [os.path.join(feed_pictures, file) for file in os.listdir(feed_pictures) if file.endswith(".jpg") or file.endswith(".png")]
    box_files = [os.path.join(path_box, file) for file in os.listdir(path_box) if file.endswith(".txt")]
    count = 1

    for image_file, box_file in zip(image_files, box_files):
        
        image = Image.open(image_file)
        box = open(box_file, "r")
        box.FileName = os.path.basename(box_file)
        print(box.FileName)
        box_lines = box.readlines()
        parsed_boxes = []
        for line in box_lines:
            line = line.strip()
            bbox, text  = split_after_eighth_comma(line,box.FileName)
            parsed_boxes.append({"text": text, "bbox": bbox})

     
        # For example, you can access its properties like width and height
        width, height = image.size
        img_size = (width, height)
        
        # Once you are done with the image, don't forget to close it
        full_output_path = os.path.join(output, box.FileName)
    
        with open(full_output_path, 'w') as output_file:
            output_file.write(to_prompt(parsed_boxes, img_size))
            print(count)
        count+=1
        image.close()
        box.close()
latin_runner()

X00016469612.txt
1
X00016469619.txt
2
X00016469620.txt
3
X00016469622.txt
4
X00016469623.txt
5
X00016469669.txt
6
X00016469670.txt
7
X00016469671.txt
8
X00016469672.txt
9
X00016469676.txt
10
X51005200931.txt
11
X51005200938.txt
12
X51005230605.txt
13
X51005230616.txt
14
X51005230617.txt
15
X51005230621.txt
16
X51005230648.txt
17
X51005230657.txt
18
X51005230659.txt
19
X51005255805.txt
20
X51005268200.txt
21
X51005268262.txt
22
X51005268275.txt
23
X51005268400.txt
24
X51005268408.txt
25
X51005268472.txt
26
X51005288570.txt
27
X51005301659.txt
28
X51005301661.txt
29
X51005301666.txt
30
X51005301667.txt
31
X51005303661.txt
32
X51005306399.txt
33
X51005337867.txt
34
X51005337872.txt
35
X51005337877.txt
36
X51005361883.txt
37
X51005361895.txt
38
X51005361897.txt
39
X51005361898.txt
40
X51005361900.txt
41
X51005361906.txt
42
X51005361907.txt
43
X51005361908.txt
44
X51005361912.txt
45
X51005361923.txt
46
X51005361946.txt
47
X51005361950.txt
48
X51005365179.txt
49
X51005365187.txt
50
X51005433

Remove the LATIN layouts from the output folder and safe for future use.
Move the cleaned prices from the feed into another folder, they will later be used as the ground truth.  
Move the OCR data into the feed folder

In [26]:
def remove_box():
  input_folder = feed
  output_folder = output

  for filename in os.listdir(input_folder):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    print(filename)
    with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
      for line in infile:
        try:
          line = line.strip()
          parts = line.split(',')
          if len(parts) > 8:
            # Rejoin the parts after the first 8 items
            new_line = ','.join(parts[8:])
            # Write the modified line to the output file
            outfile.write(new_line + '\n')
          else:
            # Write the line as it is if it doesn't have 8 numbers
            outfile.write(line)
        except UnicodeDecodeError as e:
          print(f"Error processing line: {line}")
          print(f"file: {filename}")
remove_box()


X00016469612.txt
X00016469619.txt
X00016469620.txt
X00016469622.txt
X00016469623.txt
X00016469669.txt
X00016469670.txt
X00016469671.txt
X00016469672.txt
X00016469676.txt
X51005200931.txt
X51005200938.txt
X51005230605.txt
X51005230616.txt
X51005230617.txt
X51005230621.txt
X51005230648.txt
X51005230657.txt
X51005230659.txt
X51005255805.txt
X51005268200.txt
X51005268262.txt
X51005268275.txt
X51005268400.txt
X51005268408.txt
X51005268472.txt
X51005288570.txt
X51005301659.txt
X51005301661.txt
X51005301666.txt
X51005301667.txt
X51005303661.txt
X51005306399.txt
X51005337867.txt
X51005337872.txt
X51005337877.txt
X51005361883.txt
X51005361895.txt
X51005361897.txt
X51005361898.txt
X51005361900.txt
X51005361906.txt
X51005361907.txt
X51005361908.txt
X51005361912.txt
X51005361923.txt
X51005361946.txt
X51005361950.txt
X51005365179.txt
X51005365187.txt
X51005433492.txt
X51005433494.txt
X51005433514.txt
X51005433518.txt
X51005433522.txt
X51005433533.txt
X51005433538.txt
X51005433541.txt
X51005433543.t

Store the data in the output folder in a separate folder. This are the plain text instruction and extraction documents. Best move a copy into the feed_text folder.  
Replace the entries in the feed folder with the entries in the output folder.

In [27]:
def choose_50():

  to_write_to = output

  all_files = [f for f in os.listdir(feed) if os.path.isfile(os.path.join(feed, f))]
  selected_files = random.sample(all_files, 50)

  for file_name in selected_files:
    source_file_path = os.path.join(feed, file_name)
    destination_file_path = os.path.join(to_write_to, file_name)
    shutil.move(source_file_path, destination_file_path)

choose_50()

Save the remaining files of the feed folder in a separate folder. They will be used in the future for the ground truth.
Move the entries in the output folder into the feed folder and save them additionally save them in a folder. They will be used for the solutions of the instructions.

In [28]:
def get_matching_pictures():
        
  txt_files = {os.path.splitext(f)[0] for f in os.listdir(feed) if os.path.isfile(os.path.join(feed, f)) and f.endswith('.txt')}
      
  for png_file in os.listdir(feed_pictures):
    print(png_file)
    if png_file.endswith('.jpg'):
        png_file_base = os.path.splitext(png_file)[0]
        if png_file_base in txt_files:
            # Copy the matching png file to the destination folder
            source_file_path = os.path.join(feed_pictures, png_file)
            destination_file_path = os.path.join(output, png_file)
            shutil.move(source_file_path, destination_file_path)
            print(f"Moved picture file {png_file} to output folder")
get_matching_pictures()

X00016469612.jpg
X00016469619.jpg
X00016469620.jpg
X00016469622.jpg
X00016469623.jpg
X00016469669.jpg
X00016469670.jpg
X00016469671.jpg
X00016469672.jpg
X00016469676.jpg
X51005200931.jpg
X51005200938.jpg
X51005230605.jpg
X51005230616.jpg
X51005230617.jpg
X51005230621.jpg
X51005230648.jpg
X51005230657.jpg
X51005230659.jpg
X51005255805.jpg
X51005268200.jpg
X51005268262.jpg
X51005268275.jpg
X51005268400.jpg
X51005268408.jpg
X51005268472.jpg
X51005288570.jpg
X51005301659.jpg
X51005301661.jpg
X51005301666.jpg
X51005301667.jpg
X51005303661.jpg
X51005306399.jpg
X51005337867.jpg
X51005337872.jpg
X51005337877.jpg
X51005361883.jpg
X51005361895.jpg
X51005361897.jpg
X51005361898.jpg
X51005361900.jpg
X51005361906.jpg
X51005361907.jpg
X51005361908.jpg
X51005361912.jpg
X51005361923.jpg
X51005361946.jpg
X51005361950.jpg
X51005365179.jpg
X51005365187.jpg
X51005433492.jpg
X51005433494.jpg
X51005433514.jpg
X51005433518.jpg
X51005433522.jpg
X51005433533.jpg
X51005433538.jpg
X51005433541.jpg
X51005433543.j

Remember, that the remaining files in feedPictures are now the pictures used for the extraction part of the experiment. Maybe move it into a designated folder to remember.  
Move the files in the output folder into a folder for future use. This are the images used for the instructions in the future.  
Move the files in the LATIN folder (the ones converted prior) into the feed_latin folder.

In [31]:
def get_matching_latin():
        
  txt_files = {os.path.splitext(f)[0] for f in os.listdir(feed) if os.path.isfile(os.path.join(feed, f)) and f.endswith('.txt')}
      
  for latin_file in os.listdir(feed_latin):
      png_file_base = os.path.splitext(latin_file)[0]
      if png_file_base in txt_files:
            # Copy the matching png file to the destination folder
            source_file_path = os.path.join(feed_latin, latin_file)
            destination_file_path = os.path.join(output, latin_file)
            shutil.move(source_file_path, destination_file_path)
            print(f"Moved picture file {latin_file} to output folder")
get_matching_latin()

Moved picture file X51005453801.txt to output folder
Moved picture file X51005676545.txt to output folder
Moved picture file X51005676549.txt to output folder
Moved picture file X51005705727.txt to output folder
Moved picture file X51005711442.txt to output folder
Moved picture file X51005711445.txt to output folder
Moved picture file X51005724622.txt to output folder
Moved picture file X51005757248.txt to output folder
Moved picture file X51005806703.txt to output folder
Moved picture file X51005889296.txt to output folder
Moved picture file X51006008091.txt to output folder
Moved picture file X51006311778.txt to output folder
Moved picture file X51006328913.txt to output folder
Moved picture file X51006332575.txt to output folder
Moved picture file X51006334742.txt to output folder
Moved picture file X51006335552.txt to output folder
Moved picture file X51006387812.txt to output folder
Moved picture file X51006389963.txt to output folder
Moved picture file X51006414532.txt to output 

Keep the remaining files in the feed_latin folder. You might want to move it into a designated folder. They will be the extraction documents in LATIN format.   
Store the files in the output folder in a designated folder. These are the instruction documents in LATIN layout.  
Move the box data into the feed_text folder

In [36]:
def get_matching_OCR():
        
  txt_files = {os.path.splitext(f)[0] for f in os.listdir(feed) if os.path.isfile(os.path.join(feed, f)) and f.endswith('.txt')}
      
  for text_file in os.listdir(feed_text):
    png_file_base = os.path.splitext(text_file)[0]
    if png_file_base in txt_files:
      # Copy the matching png file to the destination folder
      source_file_path = os.path.join(feed_text, text_file)
      destination_file_path = os.path.join(output, text_file)
      shutil.move(source_file_path, destination_file_path)
      print(f"Moved picture file {text_file} to output folder")

get_matching_OCR()

Moved picture file X51005453801.txt to output folder
Moved picture file X51005676545.txt to output folder
Moved picture file X51005676549.txt to output folder
Moved picture file X51005705727.txt to output folder
Moved picture file X51005711442.txt to output folder
Moved picture file X51005711445.txt to output folder
Moved picture file X51005724622.txt to output folder
Moved picture file X51005757248.txt to output folder
Moved picture file X51005806703.txt to output folder
Moved picture file X51005889296.txt to output folder
Moved picture file X51006008091.txt to output folder
Moved picture file X51006311778.txt to output folder
Moved picture file X51006328913.txt to output folder
Moved picture file X51006332575.txt to output folder
Moved picture file X51006334742.txt to output folder
Moved picture file X51006335552.txt to output folder
Moved picture file X51006387812.txt to output folder
Moved picture file X51006389963.txt to output folder
Moved picture file X51006414532.txt to output 

Store the result in the output folder as the instruction documents for plain text instruction prompts and the remaining files in the feed_text folder as the extraction documents for plain text instructions.